# Spotify API Explorer - Public Data Only
Explore everything available from Spotify's public API without user login.
Extract all possible data from playlists, songs, artists, albums, and users.

## 1. Setup & Authentication (No User Login Required)

In [1]:
# Install dependencies
%pip install spotipy python-dotenv pandas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 354.2/354.2 kB 15.9 MB/s eta 0:00:00
  Using cached async_timeout-5.0.1-py3-none-any.whl (6.2 kB)

[notice] A new release of pip available: 22.3.1 -> 25.3
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [33]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd
import json
import os
from dotenv import load_dotenv
from datetime import datetime

# Load environment variables
load_dotenv()

# Initialize Spotify API with Client Credentials (no user login)
client_credentials_manager = SpotifyClientCredentials(
    client_id=os.getenv('SPOTIFY_CLIENT_ID'),
    client_secret=os.getenv('SPOTIFY_CLIENT_SECRET')
)

sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

print("✅ Spotify API initialized (Public access only - no user login)")

✅ Spotify API initialized (Public access only - no user login)


In [41]:
# ========================================
# CONFIGURATION VARIABLES
# ========================================

# Playlist to explore (URL or URI or name to search)
PLAYLIST_IDENTIFIER = "https://open.spotify.com/playlist/3XyDvjoxiae0oWpfJ4kga9?si=d2f57623799b4ebb"  # Can be name, URL, or spotify:playlist:37i9dQZF1DXcBWIGoYBM5M

# Search queries for exploration
SEARCH_SONG_NAME = "Blinding Lights"
SEARCH_ARTIST_NAME = "The Weeknd"
SEARCH_ALBUM_NAME = "After Hours"

# Limits
MAX_TRACKS_FROM_PLAYLIST = 100
MAX_SEARCH_RESULTS = 10

print("Configuration:")
print(f"  Playlist: {PLAYLIST_IDENTIFIER}")
print(f"  Search Song: {SEARCH_SONG_NAME}")
print(f"  Search Artist: {SEARCH_ARTIST_NAME}")
print(f"  Max Tracks: {MAX_TRACKS_FROM_PLAYLIST}")

Configuration:
  Playlist: https://open.spotify.com/playlist/3XyDvjoxiae0oWpfJ4kga9?si=d2f57623799b4ebb
  Search Song: Blinding Lights
  Search Artist: The Weeknd
  Max Tracks: 100


## 2. Helper Functions - Extract ALL Data

In [42]:
def extract_track_data(track):
    """Extract ALL possible data from a track."""
    
    # Get audio features (if available)
    # Note: Audio features may not be available with Client Credentials auth
    audio_features = None
    try:
        audio_features = sp.audio_features(track['id'])[0] if track.get('id') else None
    except Exception as e:
        # Silently skip - audio features require user auth for some endpoints
        pass
    
    # Get audio analysis (detailed)
    # Note: Audio analysis may not be available with Client Credentials auth
    audio_analysis = None
    try:
        audio_analysis_full = sp.audio_analysis(track['id']) if track.get('id') else None
        if audio_analysis_full:
            # Summarize the analysis (it's huge)
            audio_analysis = {
                'num_sections': len(audio_analysis_full.get('sections', [])),
                'num_segments': len(audio_analysis_full.get('segments', [])),
                'num_bars': len(audio_analysis_full.get('bars', [])),
                'num_beats': len(audio_analysis_full.get('beats', [])),
                'num_tatums': len(audio_analysis_full.get('tatums', [])),
            }
    except Exception as e:
        # Silently skip - audio analysis requires user auth for some endpoints
        pass
    
    return {
        # Basic Info
        'id': track.get('id'),
        'name': track.get('name'),
        'uri': track.get('uri'),
        'href': track.get('href'),
        'external_urls': track.get('external_urls', {}).get('spotify'),
        'preview_url': track.get('preview_url'),
        
        # Track Details
        'duration_ms': track.get('duration_ms'),
        'duration_readable': f"{track.get('duration_ms', 0) // 60000}:{(track.get('duration_ms', 0) % 60000) // 1000:02d}",
        'explicit': track.get('explicit'),
        'disc_number': track.get('disc_number'),
        'track_number': track.get('track_number'),
        'popularity': track.get('popularity'),
        'is_local': track.get('is_local'),
        'is_playable': track.get('is_playable'),
        
        # Album Info
        'album_id': track.get('album', {}).get('id'),
        'album_name': track.get('album', {}).get('name'),
        'album_type': track.get('album', {}).get('album_type'),
        'album_release_date': track.get('album', {}).get('release_date'),
        'album_release_date_precision': track.get('album', {}).get('release_date_precision'),
        'album_total_tracks': track.get('album', {}).get('total_tracks'),
        'album_uri': track.get('album', {}).get('uri'),
        'album_external_url': track.get('album', {}).get('external_urls', {}).get('spotify'),
        'album_images': [{'url': img.get('url'), 'height': img.get('height'), 'width': img.get('width')} 
                        for img in track.get('album', {}).get('images', [])],
        
        # Artists Info
        'artists': [
            {
                'id': artist.get('id'),
                'name': artist.get('name'),
                'uri': artist.get('uri'),
                'external_url': artist.get('external_urls', {}).get('spotify')
            }
            for artist in track.get('artists', [])
        ],
        'artists_names': ', '.join([a.get('name', '') for a in track.get('artists', [])]),
        
        # Available Markets
        'available_markets': track.get('available_markets', []),
        'num_available_markets': len(track.get('available_markets', [])),
        
        # Audio Features (if available) - May be None with Client Credentials
        'audio_features': audio_features,
        
        # Audio Analysis Summary - May be None with Client Credentials
        'audio_analysis': audio_analysis,
    }


def extract_artist_full_data(artist_id):
    """Get FULL artist data including top tracks, albums, and related artists."""
    try:
        artist = sp.artist(artist_id)
        top_tracks = sp.artist_top_tracks(artist_id)
        albums = sp.artist_albums(artist_id, limit=20)
        related = sp.artist_related_artists(artist_id)
        
        return {
            # Basic Info
            'id': artist.get('id'),
            'name': artist.get('name'),
            'uri': artist.get('uri'),
            'href': artist.get('href'),
            'external_urls': artist.get('external_urls', {}).get('spotify'),
            
            # Popularity & Followers
            'popularity': artist.get('popularity'),
            'followers': artist.get('followers', {}).get('total'),
            
            # Genres
            'genres': artist.get('genres', []),
            
            # Images
            'images': [{'url': img.get('url'), 'height': img.get('height'), 'width': img.get('width')} 
                      for img in artist.get('images', [])],
            
            # Top Tracks
            'top_tracks': [
                {
                    'name': t.get('name'),
                    'popularity': t.get('popularity'),
                    'preview_url': t.get('preview_url'),
                    'uri': t.get('uri')
                }
                for t in top_tracks.get('tracks', [])[:5]
            ],
            
            # Albums Count
            'albums_count': albums.get('total'),
            'albums_sample': [
                {
                    'name': a.get('name'),
                    'release_date': a.get('release_date'),
                    'total_tracks': a.get('total_tracks'),
                    'uri': a.get('uri')
                }
                for a in albums.get('items', [])[:5]
            ],
            
            # Related Artists
            'related_artists': [
                {
                    'name': a.get('name'),
                    'popularity': a.get('popularity'),
                    'genres': a.get('genres', []),
                    'uri': a.get('uri')
                }
                for a in related.get('artists', [])[:5]
            ]
        }
    except Exception as e:
        print(f"Error getting artist data: {e}")
        return None


def extract_playlist_data(playlist_id):
    """Extract ALL playlist data."""
    try:
        playlist = sp.playlist(playlist_id)
        
        return {
            # Basic Info
            'id': playlist.get('id'),
            'name': playlist.get('name'),
            'description': playlist.get('description'),
            'uri': playlist.get('uri'),
            'href': playlist.get('href'),
            'external_urls': playlist.get('external_urls', {}).get('spotify'),
            
            # Metadata
            'collaborative': playlist.get('collaborative'),
            'public': playlist.get('public'),
            'snapshot_id': playlist.get('snapshot_id'),
            
            # Owner
            'owner': {
                'id': playlist.get('owner', {}).get('id'),
                'display_name': playlist.get('owner', {}).get('display_name'),
                'uri': playlist.get('owner', {}).get('uri'),
                'external_url': playlist.get('owner', {}).get('external_urls', {}).get('spotify')
            },
            
            # Stats
            'followers': playlist.get('followers', {}).get('total'),
            'total_tracks': playlist.get('tracks', {}).get('total'),
            
            # Images
            'images': [{'url': img.get('url'), 'height': img.get('height'), 'width': img.get('width')} 
                      for img in playlist.get('images', [])],
        }
    except Exception as e:
        print(f"Error getting playlist data: {e}")
        return None


print("✅ Helper functions defined")
print("ℹ️  Note: Audio features/analysis may not be available (requires user authentication)")

✅ Helper functions defined
ℹ️  Note: Audio features/analysis may not be available (requires user authentication)


## 3. Find Playlist (Search or Direct)

In [44]:
def get_playlist_id(identifier):
    """Get playlist ID from name, URL, or URI."""
    
    # If it's a URL
    if 'open.spotify.com/playlist/' in identifier:
        return identifier.split('playlist/')[1].split('?')[0]
    
    # If it's a URI
    if identifier.startswith('spotify:playlist:'):
        return identifier.split(':')[-1]
    
    # Otherwise search for it
    try:
        results = sp.search(q=identifier, type='playlist', limit=5)
        
        if results and results.get('playlists') and results['playlists'].get('items'):
            print(f"\n🔍 Found {len(results['playlists']['items'])} playlists matching '{identifier}':\n")
            for idx, pl in enumerate(results['playlists']['items'], 1):
                print(f"  {idx}. {pl['name']}")
                print(f"     Owner: {pl['owner']['display_name']}")
                print(f"     Tracks: {pl['tracks']['total']}")
                print(f"     ID: {pl['id']}\n")
            
            # Return first match
            return results['playlists']['items'][0]['id']
    except Exception as e:
        print(f"❌ Error searching for playlist: {e}")
        print("⚠️  Make sure you have valid SPOTIFY_CLIENT_ID and SPOTIFY_CLIENT_SECRET in your .env file")
        print("    Get credentials from: https://developer.spotify.com/dashboard")
    
    return None


# Get playlist ID
playlist_id = get_playlist_id(PLAYLIST_IDENTIFIER)

if playlist_id:
    print(f"✅ Using playlist ID: {playlist_id}")
else:
    print("❌ Playlist not found")

✅ Using playlist ID: 3XyDvjoxiae0oWpfJ4kga9


## 4. Extract ALL Playlist Data

In [45]:
if playlist_id:
    print("📋 Extracting playlist data...\n")
    
    playlist_data = extract_playlist_data(playlist_id)
    
    if playlist_data:
        print("="*80)
        print("PLAYLIST INFORMATION")
        print("="*80)
        print(f"Name: {playlist_data['name']}")
        print(f"Description: {playlist_data['description']}")
        print(f"Owner: {playlist_data['owner']['display_name']} ({playlist_data['owner']['id']})")
        print(f"Total Tracks: {playlist_data['total_tracks']}")
        print(f"Followers: {playlist_data['followers']:,}")
        print(f"Public: {playlist_data['public']}")
        print(f"Collaborative: {playlist_data['collaborative']}")
        print(f"URL: {playlist_data['external_urls']}")
        print("="*80)
        
        # Display as JSON
        print("\n📄 Full Playlist JSON:\n")
        print(json.dumps(playlist_data, indent=2)[:2000] + "...")
    else:
        print("❌ Failed to extract playlist data. Check your API credentials in .env file.")
else:
    playlist_data = None
    print("❌ No playlist to extract")

📋 Extracting playlist data...

PLAYLIST INFORMATION
Name: Summer Nights
Description: 
Owner: Malik (1320_music)
Total Tracks: 67
Followers: 0
Public: True
Collaborative: False
URL: https://open.spotify.com/playlist/3XyDvjoxiae0oWpfJ4kga9

📄 Full Playlist JSON:

{
  "id": "3XyDvjoxiae0oWpfJ4kga9",
  "name": "Summer Nights",
  "description": "",
  "uri": "spotify:playlist:3XyDvjoxiae0oWpfJ4kga9",
  "href": "https://api.spotify.com/v1/playlists/3XyDvjoxiae0oWpfJ4kga9?additional_types=track",
  "external_urls": "https://open.spotify.com/playlist/3XyDvjoxiae0oWpfJ4kga9",
  "collaborative": false,
  "public": true,
  "snapshot_id": "AAAASKL6MYZ8K2BLe/H7cjwBtubSVmrQ",
  "owner": {
    "id": "1320_music",
    "display_name": "Malik",
    "uri": "spotify:user:1320_music",
    "external_url": "https://open.spotify.com/user/1320_music"
  },
  "followers": 0,
  "total_tracks": 67,
  "images": [
    {
      "url": "https://image-cdn-fa.spotifycdn.com/image/ab67706c0000da848f15c90b02dadc6c6c154bb4",

## 5. Extract ALL Tracks from Playlist

In [46]:
if playlist_id:
    print("🎵 Extracting tracks from playlist...\n")
    
    # Get playlist tracks
    results = sp.playlist_tracks(playlist_id, limit=MAX_TRACKS_FROM_PLAYLIST)
    
    all_tracks_data = []
    
    for idx, item in enumerate(results['items'], 1):
        track = item['track']
        if track:  # Sometimes tracks can be None
            print(f"[{idx}/{len(results['items'])}] Processing: {track['name']} by {track['artists'][0]['name']}")
            
            track_data = extract_track_data(track)
            
            # Add playlist-specific info
            track_data['added_at'] = item.get('added_at')
            track_data['added_by'] = item.get('added_by', {}).get('id')
            
            all_tracks_data.append(track_data)
    
    print(f"\n✅ Extracted {len(all_tracks_data)} tracks with full data")
else:
    all_tracks_data = []
    print("❌ No playlist to extract tracks from")

🎵 Extracting tracks from playlist...



HTTP Error for GET to https://api.spotify.com/v1/audio-features/?ids=5i7rT8lbGzjj1n7TTXR5U8 with Params: {} returned 403 due to None
HTTP Error for GET to https://api.spotify.com/v1/audio-analysis/5i7rT8lbGzjj1n7TTXR5U8 with Params: {} returned 403 due to None
HTTP Error for GET to https://api.spotify.com/v1/audio-features/?ids=2wnsBaxrmkthIFAm6vqCuX with Params: {} returned 403 due to None
HTTP Error for GET to https://api.spotify.com/v1/audio-analysis/2wnsBaxrmkthIFAm6vqCuX with Params: {} returned 403 due to None


[1/67] Processing: Everlasting Love by Carl Carlton
[2/67] Processing: So Lonely by The Police
[3/67] Processing: I Only Have Eyes for You by The Flamingos


HTTP Error for GET to https://api.spotify.com/v1/audio-features/?ids=3YdKJzcoMZMacISlpY4QoP with Params: {} returned 403 due to None
HTTP Error for GET to https://api.spotify.com/v1/audio-analysis/3YdKJzcoMZMacISlpY4QoP with Params: {} returned 403 due to None
HTTP Error for GET to https://api.spotify.com/v1/audio-features/?ids=6Y3h8CAOKtFSKGN1hTRA1D with Params: {} returned 403 due to None
HTTP Error for GET to https://api.spotify.com/v1/audio-analysis/6Y3h8CAOKtFSKGN1hTRA1D with Params: {} returned 403 due to None
HTTP Error for GET to https://api.spotify.com/v1/audio-features/?ids=46LEQkbfPT5x2ZanS3STg5 with Params: {} returned 403 due to None
HTTP Error for GET to https://api.spotify.com/v1/audio-analysis/46LEQkbfPT5x2ZanS3STg5 with Params: {} returned 403 due to None
HTTP Error for GET to https://api.spotify.com/v1/audio-features/?ids=4nZi6XNe36Ut4Nij3IQ1yC with Params: {} returned 403 due to None


[4/67] Processing: I Never Thought I'd See the Day by Sade
[5/67] Processing: I'm Still In Love With You by New Edition
[6/67] Processing: Silver Springs - 2004 Remaster by Fleetwood Mac


...

[62/67] Processing: Wait by Maroon 5
[63/67] Processing: I Know What You Did Last Summer by Shawn Mendes
[64/67] Processing: I Don't Wanna Know by Charli xcx


HTTP Error for GET to https://api.spotify.com/v1/audio-analysis/2wsypbBdFwN1woTAh9sq6X with Params: {} returned 403 due to None
HTTP Error for GET to https://api.spotify.com/v1/audio-features/?ids=64xw3gXNMWv4wNXjF6tnXr with Params: {} returned 403 due to None
HTTP Error for GET to https://api.spotify.com/v1/audio-analysis/64xw3gXNMWv4wNXjF6tnXr with Params: {} returned 403 due to None
HTTP Error for GET to https://api.spotify.com/v1/audio-features/?ids=3EheMxGtUUePA3cHDq2nKv with Params: {} returned 403 due to None
HTTP Error for GET to https://api.spotify.com/v1/audio-analysis/3EheMxGtUUePA3cHDq2nKv with Params: {} returned 403 due to None


[65/67] Processing: Tears (feat. Caroline Polachek) by Charli xcx
[66/67] Processing: Shooting Star by Carly Rae Jepsen
[67/67] Processing: Comeback by Carly Rae Jepsen


HTTP Error for GET to https://api.spotify.com/v1/audio-features/?ids=3wdrHPqEs2U5lhhDwked0A with Params: {} returned 403 due to None
HTTP Error for GET to https://api.spotify.com/v1/audio-analysis/3wdrHPqEs2U5lhhDwked0A with Params: {} returned 403 due to None



✅ Extracted 67 tracks with full data


## 6. View Tracks as DataFrame

In [47]:
if all_tracks_data:
    # Create simplified version for DataFrame (remove nested objects)
    tracks_for_df = []
    
    for track in all_tracks_data:
        simplified = {
            'name': track['name'],
            'artists': track['artists_names'],
            'album': track['album_name'],
            'duration': track['duration_readable'],
            'popularity': track['popularity'],
            'explicit': track['explicit'],
            'release_date': track['album_release_date'],
            'preview_url': track['preview_url'],
        }
        
        # Add audio features if available
        if track.get('audio_features'):
            af = track['audio_features']
            simplified.update({
                'tempo': af.get('tempo'),
                'key': af.get('key'),
                'mode': af.get('mode'),
                'danceability': af.get('danceability'),
                'energy': af.get('energy'),
                'valence': af.get('valence'),
                'acousticness': af.get('acousticness'),
                'instrumentalness': af.get('instrumentalness'),
                'speechiness': af.get('speechiness'),
            })
        
        tracks_for_df.append(simplified)
    
    tracks_df = pd.DataFrame(tracks_for_df)
    
    print(f"📊 Tracks Overview ({len(tracks_df)} tracks):\n")
    display(tracks_df)
else:
    print("No tracks to display")

📊 Tracks Overview (67 tracks):



,name,artists,album,duration,popularity,explicit,release_date,preview_url
0,Everlasting Love,Carl Carlton,Everlasting: The Best Of Carl Carlton,2:33,55,False,2009-01-01,None
1,So Lonely,The Police,Outlandos D'Amour (Remastered 2003),4:49,69,False,1978-11-02,None
2,I Only Have Eyes for You,The Flamingos,Flamingo Serenade,3:22,73,False,1959-08-11,None
3,I Never Thought I'd See the Day,Sade,Stronger Than Pride,4:13,55,False,1988-05-03,None
4,I'm Still In Love With You,New Edition,Home Again,4:39,60,False,1996-01-01,None
...,...,...,...,...,...,...,...,...
62,I Know What You Did Last Summer,"Shawn Mendes, Camila Cabello",Handwritten,3:43,75,False,2015-04-14,None
63,I Don't Wanna Know,Charli xcx,Charli,3:05,41,False,2019-09-13,None
64,Tears (feat. Caroline Polachek),"Charli xcx, Caroline Polachek",Pop 2,4:13,53,False,2017-12-15,None
65,Shooting Star,Carly Rae Jepsen,The Loneliest Time,3:19,11,False,2022-10-21,None


## 7. Show Full Track JSON Example

In [48]:
if all_tracks_data:
    print("📄 Example: Full Track Data (First Track):\n")
    print(json.dumps(all_tracks_data[0], indent=2))
else:
    print("No track data to display")

📄 Example: Full Track Data (First Track):

{
  "id": "5i7rT8lbGzjj1n7TTXR5U8",
  "name": "Everlasting Love",
  "uri": "spotify:track:5i7rT8lbGzjj1n7TTXR5U8",
  "href": "https://api.spotify.com/v1/tracks/5i7rT8lbGzjj1n7TTXR5U8",
  "external_urls": "https://open.spotify.com/track/5i7rT8lbGzjj1n7TTXR5U8",
  "preview_url": null,
  "duration_ms": 153946,
  "duration_readable": "2:33",
  "explicit": false,
  "disc_number": 1,
  "track_number": 14,
  "popularity": 55,
  "is_local": false,
  "is_playable": null,
  "album_id": "10np3FVXIDYUca9O6bd5wb",
  "album_name": "Everlasting: The Best Of Carl Carlton",
  "album_type": "compilation",
  "album_release_date": "2009-01-01",
  "album_release_date_precision": "day",
  "album_total_tracks": 22,
  "album_uri": "spotify:album:10np3FVXIDYUca9O6bd5wb",
  "album_external_url": "https://open.spotify.com/album/10np3FVXIDYUca9O6bd5wb",
  "album_images": [
    {
      "url": "https://i.scdn.co/image/ab67616d0000b273117677a6ce6415dc5b70608c",
      "heigh

## 8. Search: Does Song Exist?

In [49]:
print(f"🔍 Searching for song: '{SEARCH_SONG_NAME}'\n")

# Search for track
track_results = sp.search(q=SEARCH_SONG_NAME, type='track', limit=MAX_SEARCH_RESULTS)

if track_results['tracks']['items']:
    print(f"✅ Found {len(track_results['tracks']['items'])} tracks:\n")
    
    search_tracks = []
    
    for idx, track in enumerate(track_results['tracks']['items'], 1):
        print(f"{idx}. {track['name']}")
        print(f"   Artist: {', '.join([a['name'] for a in track['artists']])}")
        print(f"   Album: {track['album']['name']}")
        print(f"   Popularity: {track['popularity']}/100")
        print(f"   Release: {track['album']['release_date']}")
        print(f"   URI: {track['uri']}")
        print()
        
        # Extract full data
        track_full = extract_track_data(track)
        search_tracks.append(track_full)
    
    print(f"✅ Extracted full data for {len(search_tracks)} matching tracks")
else:
    search_tracks = []
    print(f"❌ No tracks found matching '{SEARCH_SONG_NAME}'")

🔍 Searching for song: 'Blinding Lights'



HTTP Error for GET to https://api.spotify.com/v1/audio-features/?ids=0VjIjW4GlUZAMYd2vXMi3b with Params: {} returned 403 due to None
HTTP Error for GET to https://api.spotify.com/v1/audio-analysis/0VjIjW4GlUZAMYd2vXMi3b with Params: {} returned 403 due to None
HTTP Error for GET to https://api.spotify.com/v1/audio-features/?ids=66aMnwdhkEc1V0AAnyYchX with Params: {} returned 403 due to None
HTTP Error for GET to https://api.spotify.com/v1/audio-analysis/66aMnwdhkEc1V0AAnyYchX with Params: {} returned 403 due to None


✅ Found 10 tracks:

1. Blinding Lights
   Artist: The Weeknd
   Album: After Hours
   Popularity: 91/100
   Release: 2020-03-20
   URI: spotify:track:0VjIjW4GlUZAMYd2vXMi3b

2. Blinding Lights
   Artist: Teddy Swims
   Album: Blinding Lights
   Popularity: 67/100
   Release: 2020-05-15
   URI: spotify:track:66aMnwdhkEc1V0AAnyYchX

3. Blinding Lights
   Artist: Adrienne Clotard
   Album: Blinding Lights
   Popularity: 57/100
   Release: 2024-11-01
   URI: spotify:track:0lyj9SQRKOg5hEOUzkmwZd



HTTP Error for GET to https://api.spotify.com/v1/audio-features/?ids=0lyj9SQRKOg5hEOUzkmwZd with Params: {} returned 403 due to None
HTTP Error for GET to https://api.spotify.com/v1/audio-analysis/0lyj9SQRKOg5hEOUzkmwZd with Params: {} returned 403 due to None
HTTP Error for GET to https://api.spotify.com/v1/audio-features/?ids=0cQJGoRki2avKVodXEmr0d with Params: {} returned 403 due to None
HTTP Error for GET to https://api.spotify.com/v1/audio-analysis/0cQJGoRki2avKVodXEmr0d with Params: {} returned 403 due to None
HTTP Error for GET to https://api.spotify.com/v1/audio-features/?ids=6cRJTmba0JHDquftAkxUgG with Params: {} returned 403 due to None
HTTP Error for GET to https://api.spotify.com/v1/audio-analysis/6cRJTmba0JHDquftAkxUgG with Params: {} returned 403 due to None


4. Blinding Lights
   Artist: Loi
   Album: Blinding Lights
   Popularity: 61/100
   Release: 2021-09-17
   URI: spotify:track:0cQJGoRki2avKVodXEmr0d

5. Blinding Lights - Country Version
   Artist: Tebey
   Album: Blinding Lights (Country Version)
   Popularity: 52/100
   Release: 2024-03-09
   URI: spotify:track:6cRJTmba0JHDquftAkxUgG

6. Blinding Lights
   Artist: The Weeknd
   Album: Blinding Lights
   Popularity: 62/100
   Release: 2020-09-11
   URI: spotify:track:6qYkmqFsXbj8CQjAdbYz07



HTTP Error for GET to https://api.spotify.com/v1/audio-features/?ids=6qYkmqFsXbj8CQjAdbYz07 with Params: {} returned 403 due to None
HTTP Error for GET to https://api.spotify.com/v1/audio-analysis/6qYkmqFsXbj8CQjAdbYz07 with Params: {} returned 403 due to None
HTTP Error for GET to https://api.spotify.com/v1/audio-features/?ids=2uJw7yR61snp7s39YmUyhf with Params: {} returned 403 due to None
HTTP Error for GET to https://api.spotify.com/v1/audio-analysis/2uJw7yR61snp7s39YmUyhf with Params: {} returned 403 due to None
HTTP Error for GET to https://api.spotify.com/v1/audio-features/?ids=1nJOzQiAY0d9YGopbYlQsr with Params: {} returned 403 due to None
HTTP Error for GET to https://api.spotify.com/v1/audio-analysis/1nJOzQiAY0d9YGopbYlQsr with Params: {} returned 403 due to None


7. Blinding Lights - Piano Version
   Artist: Oscar Lento
   Album: Blinding Memories of Somebody (Piano Version)
   Popularity: 56/100
   Release: 2025-07-07
   URI: spotify:track:2uJw7yR61snp7s39YmUyhf

8. Blinding Lights
   Artist: Saint Asonia
   Album: Blinding Lights
   Popularity: 50/100
   Release: 2021-11-18
   URI: spotify:track:1nJOzQiAY0d9YGopbYlQsr

9. Blinding Lights
   Artist: Demi Femme
   Album: Blinding Lights
   Popularity: 52/100
   Release: 2024-01-26
   URI: spotify:track:7q8Tf2APnxcQWKT69nfegF



HTTP Error for GET to https://api.spotify.com/v1/audio-features/?ids=7q8Tf2APnxcQWKT69nfegF with Params: {} returned 403 due to None
HTTP Error for GET to https://api.spotify.com/v1/audio-analysis/7q8Tf2APnxcQWKT69nfegF with Params: {} returned 403 due to None
HTTP Error for GET to https://api.spotify.com/v1/audio-features/?ids=4eQO9RHTjYT1O9DTuZLj9v with Params: {} returned 403 due to None
HTTP Error for GET to https://api.spotify.com/v1/audio-analysis/4eQO9RHTjYT1O9DTuZLj9v with Params: {} returned 403 due to None


10. Blinding Lights
   Artist: The Weeknd
   Album: After Hours (Deluxe)
   Popularity: 47/100
   Release: 2020-03-20
   URI: spotify:track:4eQO9RHTjYT1O9DTuZLj9v

✅ Extracted full data for 10 matching tracks


## 9. Search: Artist Exists? + Full Info

In [50]:
print(f"🔍 Searching for artist: '{SEARCH_ARTIST_NAME}'\n")

# Search for artist
artist_results = sp.search(q=SEARCH_ARTIST_NAME, type='artist', limit=MAX_SEARCH_RESULTS)

if artist_results['artists']['items']:
    print(f"✅ Found {len(artist_results['artists']['items'])} artists:\n")
    
    search_artists = []
    
    for idx, artist in enumerate(artist_results['artists']['items'], 1):
        print(f"{idx}. {artist['name']}")
        print(f"   Popularity: {artist['popularity']}/100")
        print(f"   Followers: {artist['followers']['total']:,}")
        print(f"   Genres: {', '.join(artist['genres']) if artist['genres'] else 'N/A'}")
        print(f"   URI: {artist['uri']}")
        print()
        
        # Get FULL artist data
        artist_full = extract_artist_full_data(artist['id'])
        if artist_full:
            search_artists.append(artist_full)
    
    print(f"✅ Extracted FULL data for {len(search_artists)} artists (including top tracks, albums, related artists)")
else:
    search_artists = []
    print(f"❌ No artists found matching '{SEARCH_ARTIST_NAME}'")

🔍 Searching for artist: 'The Weeknd'

✅ Found 10 artists:

1. The Weeknd
   Popularity: 96/100
   Followers: 115,465,453
   Genres: N/A
   URI: spotify:artist:1Xyo4u8uXC1ZmMpatF05PJ



HTTP Error for GET to https://api.spotify.com/v1/artists/1Xyo4u8uXC1ZmMpatF05PJ/related-artists with Params: {} returned 404 due to Not Found


Error getting artist data: http status: 404, code: -1 - https://api.spotify.com/v1/artists/1Xyo4u8uXC1ZmMpatF05PJ/related-artists:
 Not Found, reason: None
2. The Weeks
   Popularity: 30/100
   Followers: 41,789
   Genres: N/A
   URI: spotify:artist:3zE5jV6Uw9hhdWCXM8hS3j



HTTP Error for GET to https://api.spotify.com/v1/artists/3zE5jV6Uw9hhdWCXM8hS3j/related-artists with Params: {} returned 404 due to Not Found


Error getting artist data: http status: 404, code: -1 - https://api.spotify.com/v1/artists/3zE5jV6Uw9hhdWCXM8hS3j/related-artists:
 Not Found, reason: None
3. Post Malone
   Popularity: 88/100
   Followers: 47,956,715
   Genres: N/A
   URI: spotify:artist:246dkjvS1zLTtiykXe5h60



...

Error getting artist data: http status: 404, code: -1 - https://api.spotify.com/v1/artists/06HL4z0CvFAxyc27GXpf02/related-artists:
 Not Found, reason: None
9. The Weekend Quintet
   Popularity: 24/100
   Followers: 267
   Genres: N/A
   URI: spotify:artist:2JVLB6xMAa3vXDD89dktLZ



HTTP Error for GET to https://api.spotify.com/v1/artists/2JVLB6xMAa3vXDD89dktLZ/related-artists with Params: {} returned 404 due to Not Found


Error getting artist data: http status: 404, code: -1 - https://api.spotify.com/v1/artists/2JVLB6xMAa3vXDD89dktLZ/related-artists:
 Not Found, reason: None
10. Abel Tesfaye
   Popularity: 7/100
   Followers: 3,334
   Genres: N/A
   URI: spotify:artist:56pImxBsOQTCASzJNLNEKN



HTTP Error for GET to https://api.spotify.com/v1/artists/56pImxBsOQTCASzJNLNEKN/related-artists with Params: {} returned 404 due to Not Found


Error getting artist data: http status: 404, code: -1 - https://api.spotify.com/v1/artists/56pImxBsOQTCASzJNLNEKN/related-artists:
 Not Found, reason: None
✅ Extracted FULL data for 0 artists (including top tracks, albums, related artists)


## 10. Show Full Artist Data Example

In [51]:
if search_artists:
    print("📄 Example: Full Artist Data (First Match):\n")
    print(json.dumps(search_artists[0], indent=2))
else:
    print("No artist data to display")

No artist data to display


## 11. Search: Albums & Playlists

In [52]:
# Search for album
print(f"🔍 Searching for album: '{SEARCH_ALBUM_NAME}'\n")

album_results = sp.search(q=SEARCH_ALBUM_NAME, type='album', limit=5)

search_albums = []

if album_results['albums']['items']:
    print(f"✅ Found {len(album_results['albums']['items'])} albums:\n")
    
    for idx, album in enumerate(album_results['albums']['items'], 1):
        print(f"{idx}. {album['name']}")
        print(f"   Artist: {', '.join([a['name'] for a in album['artists']])}")
        print(f"   Release: {album['release_date']}")
        print(f"   Total Tracks: {album['total_tracks']}")
        print(f"   URI: {album['uri']}")
        print()
        
        # Get full album data
        try:
            album_full = sp.album(album['id'])
            album_data = {
                'id': album_full.get('id'),
                'name': album_full.get('name'),
                'artists': [{'name': a['name'], 'id': a['id']} for a in album_full.get('artists', [])],
                'release_date': album_full.get('release_date'),
                'total_tracks': album_full.get('total_tracks'),
                'genres': album_full.get('genres', []),
                'label': album_full.get('label'),
                'popularity': album_full.get('popularity'),
                'uri': album_full.get('uri'),
                'external_url': album_full.get('external_urls', {}).get('spotify'),
                'copyrights': album_full.get('copyrights', []),
                'tracks': [{'name': t['name'], 'duration_ms': t['duration_ms'], 'uri': t['uri']} 
                          for t in album_full.get('tracks', {}).get('items', [])]
            }
            search_albums.append(album_data)
        except Exception as e:
            print(f"   Error getting full album data: {e}")
else:
    print(f"❌ No albums found matching '{SEARCH_ALBUM_NAME}'")

🔍 Searching for album: 'After Hours'

✅ Found 5 albums:

1. After Hours
   Artist: The Weeknd
   Release: 2020-03-20
   Total Tracks: 14
   URI: spotify:album:4yP0hdKOZPNshxUOjY0cZj

2. After Hours (Deluxe)
   Artist: The Weeknd
   Release: 2020-03-20
   Total Tracks: 18
   URI: spotify:album:742eAldb4AJKLoPgJhGRE7

3. After Hours
   Artist: Kehlani
   Release: 2024-04-04
   Total Tracks: 1
   URI: spotify:album:062urW4KS66GPlEHIueefz

4. AFTER HOURS
   Artist: Avenoir
   Release: 2023-10-27
   Total Tracks: 1
   URI: spotify:album:0C4vpjaKHhAAcd0fo9hWaT

5. After Hours (Deluxe)
   Artist: The Weeknd
   Release: 2020-03-20
   Total Tracks: 18
   URI: spotify:album:4ZyeHIgdujClXSjC2cSqSb



## 12. Get User Info (Playlist Owner)

In [53]:
if playlist_data and isinstance(playlist_data, dict):
    owner_id = playlist_data['owner']['id']
    
    print(f"👤 Getting user info for playlist owner: {owner_id}\n")
    
    try:
        user = sp.user(owner_id)
        
        user_data = {
            'id': user.get('id'),
            'display_name': user.get('display_name'),
            'uri': user.get('uri'),
            'external_url': user.get('external_urls', {}).get('spotify'),
            'followers': user.get('followers', {}).get('total'),
            'images': [{'url': img.get('url'), 'height': img.get('height'), 'width': img.get('width')} 
                      for img in user.get('images', [])],
        }
        
        print("="*80)
        print("USER INFORMATION (Playlist Owner)")
        print("="*80)
        print(f"ID: {user_data['id']}")
        print(f"Display Name: {user_data['display_name']}")
        print(f"Followers: {user_data['followers']:,}" if user_data['followers'] else "Followers: N/A")
        print(f"URL: {user_data['external_url']}")
        print("="*80)
        
        print("\n📄 Full User JSON:\n")
        print(json.dumps(user_data, indent=2))
        
    except Exception as e:
        user_data = None
        print(f"❌ Error getting user data: {e}")
else:
    user_data = None
    print("No playlist owner to fetch")

👤 Getting user info for playlist owner: 1320_music

USER INFORMATION (Playlist Owner)
ID: 1320_music
Display Name: Malik
Followers: N/A
URL: https://open.spotify.com/user/1320_music

📄 Full User JSON:

{
  "id": "1320_music",
  "display_name": "Malik",
  "uri": "spotify:user:1320_music",
  "external_url": "https://open.spotify.com/user/1320_music",
  "followers": 0,
  "images": [
    {
      "url": "https://i.scdn.co/image/ab6775700000ee859322a890cbae24b92592a235",
      "height": 300,
      "width": 300
    },
    {
      "url": "https://i.scdn.co/image/ab67757000003b829322a890cbae24b92592a235",
      "height": 64,
      "width": 64
    }
  ]
}


## 13. Summary Statistics

In [54]:
print("="*80)
print("📊 SPOTIFY API EXPLORATION SUMMARY")
print("="*80)

if playlist_data and isinstance(playlist_data, dict):
    print(f"\n🎵 PLAYLIST:")
    print(f"  Name: {playlist_data['name']}")
    print(f"  Tracks Extracted: {len(all_tracks_data)}")
    print(f"  Owner: {playlist_data['owner']['display_name']}")

if all_tracks_data:
    avg_popularity = sum(t.get('popularity', 0) for t in all_tracks_data) / len(all_tracks_data)
    print(f"\n📀 TRACKS:")
    print(f"  Total: {len(all_tracks_data)}")
    print(f"  Average Popularity: {avg_popularity:.1f}/100")
    
    # Audio features stats
    with_features = [t for t in all_tracks_data if t.get('audio_features')]
    if with_features:
        avg_energy = sum(t['audio_features']['energy'] for t in with_features) / len(with_features)
        avg_danceability = sum(t['audio_features']['danceability'] for t in with_features) / len(with_features)
        avg_valence = sum(t['audio_features']['valence'] for t in with_features) / len(with_features)
        
        print(f"  Avg Energy: {avg_energy:.2f}")
        print(f"  Avg Danceability: {avg_danceability:.2f}")
        print(f"  Avg Valence (Happiness): {avg_valence:.2f}")

if search_tracks:
    print(f"\n🔍 SEARCH - TRACKS:")
    print(f"  Found: {len(search_tracks)} matches for '{SEARCH_SONG_NAME}'")

if search_artists:
    print(f"\n🎤 SEARCH - ARTISTS:")
    print(f"  Found: {len(search_artists)} matches for '{SEARCH_ARTIST_NAME}'")
    if search_artists[0].get('followers'):
        print(f"  Top Match: {search_artists[0]['name']} ({search_artists[0]['followers']:,} followers)")

if search_albums:
    print(f"\n💿 SEARCH - ALBUMS:")
    print(f"  Found: {len(search_albums)} matches for '{SEARCH_ALBUM_NAME}'")

print("\n" + "="*80)

📊 SPOTIFY API EXPLORATION SUMMARY

🎵 PLAYLIST:
  Name: Summer Nights
  Tracks Extracted: 67
  Owner: Malik

📀 TRACKS:
  Total: 67
  Average Popularity: 62.8/100

🔍 SEARCH - TRACKS:
  Found: 10 matches for 'Blinding Lights'

💿 SEARCH - ALBUMS:
  Found: 5 matches for 'After Hours'



## 14. Export All Data to JSON

In [55]:
# Create comprehensive output
output_data = {
    'metadata': {
        'scraped_at': datetime.now().isoformat(),
        'playlist_identifier': PLAYLIST_IDENTIFIER,
        'search_queries': {
            'song': SEARCH_SONG_NAME,
            'artist': SEARCH_ARTIST_NAME,
            'album': SEARCH_ALBUM_NAME
        }
    },
    'playlist': playlist_data if (playlist_data and isinstance(playlist_data, dict)) else None,
    'playlist_tracks': all_tracks_data if all_tracks_data else [],
    'user': user_data if (user_data and isinstance(user_data, dict)) else None,
    'search_results': {
        'tracks': search_tracks if search_tracks else [],
        'artists': search_artists if search_artists else [],
        'albums': search_albums if search_albums else []
    }
}

# Save to JSON
filename = f"spotify_data_{datetime.now().strftime('%Y%m%d_%H%M%S')}.json"
with open(filename, 'w', encoding='utf-8') as f:
    json.dump(output_data, f, indent=2, ensure_ascii=False)

print(f"✅ All data exported to: {filename}")
print(f"   File size: {os.path.getsize(filename) / 1024:.2f} KB")

# Also save tracks DataFrame if available
if all_tracks_data:
    csv_filename = f"spotify_tracks_{datetime.now().strftime('%Y%m%d_%H%M%S')}.csv"
    tracks_df.to_csv(csv_filename, index=False)
    print(f"✅ Tracks DataFrame exported to: {csv_filename}")

✅ All data exported to: spotify_data_20251216_220302.json
   File size: 337.03 KB
✅ Tracks DataFrame exported to: spotify_tracks_20251216_220302.csv


## 15. Reverse Lookup Examples

In [56]:
print("🔄 REVERSE LOOKUP EXAMPLES\n")
print("="*80)

# Example 1: Song name -> Does it exist?
print("\n1️⃣ Song Name → Existence Check:")
test_song = "Shape of You"
result = sp.search(q=test_song, type='track', limit=1)
if result['tracks']['items']:
    track = result['tracks']['items'][0]
    print(f"   ✅ '{test_song}' EXISTS")
    print(f"      Artist: {track['artists'][0]['name']}")
    print(f"      Popularity: {track['popularity']}/100")
    print(f"      URI: {track['uri']}")
else:
    print(f"   ❌ '{test_song}' NOT FOUND")

# Example 2: Artist name -> Get all info
print("\n2️⃣ Artist Name → Full Info:")
test_artist = "Drake"
result = sp.search(q=test_artist, type='artist', limit=1)
if result['artists']['items']:
    artist = result['artists']['items'][0]
    artist_full = extract_artist_full_data(artist['id'])
    print(f"   ✅ '{test_artist}' EXISTS")
    print(f"      Followers: {artist_full['followers']:,}")
    print(f"      Genres: {', '.join(artist_full['genres'][:3])}")
    print(f"      Top Tracks: {len(artist_full['top_tracks'])}")
    print(f"      Albums: {artist_full['albums_count']}")
    print(f"      Related Artists: {len(artist_full['related_artists'])}")
else:
    print(f"   ❌ '{test_artist}' NOT FOUND")

# Example 3: Album name -> Get tracks
print("\n3️⃣ Album Name → Get All Tracks:")
test_album = "Thriller"
result = sp.search(q=test_album, type='album', limit=1)
if result['albums']['items']:
    album = result['albums']['items'][0]
    album_full = sp.album(album['id'])
    print(f"   ✅ '{test_album}' EXISTS")
    print(f"      Artist: {album_full['artists'][0]['name']}")
    print(f"      Release: {album_full['release_date']}")
    print(f"      Tracks: {album_full['total_tracks']}")
    print(f"      First 3 tracks:")
    for idx, track in enumerate(album_full['tracks']['items'][:3], 1):
        print(f"         {idx}. {track['name']}")
else:
    print(f"   ❌ '{test_album}' NOT FOUND")

# Example 4: Playlist name -> Get full data
print("\n4️⃣ Playlist Name → Full Playlist:")
test_playlist = "Rap Caviar"
result = sp.search(q=test_playlist, type='playlist', limit=1)
if result['playlists']['items']:
    playlist = result['playlists']['items'][0]
    print(f"   ✅ '{test_playlist}' EXISTS")
    print(f"      Owner: {playlist['owner']['display_name']}")
    print(f"      Tracks: {playlist['tracks']['total']}")
    print(f"      Description: {playlist['description'][:100]}...")
else:
    print(f"   ❌ '{test_playlist}' NOT FOUND")

print("\n" + "="*80)
print("✅ All reverse lookups demonstrate: Name → Exists? → Get Full Data")

🔄 REVERSE LOOKUP EXAMPLES


1️⃣ Song Name → Existence Check:
   ✅ 'Shape of You' EXISTS
      Artist: Ed Sheeran
      Popularity: 89/100
      URI: spotify:track:7qiZfU4dY1lWllzX7mPBI3

2️⃣ Artist Name → Full Info:


HTTP Error for GET to https://api.spotify.com/v1/artists/3TVXtAsR1Inumwj472S9r4/related-artists with Params: {} returned 404 due to Not Found


Error getting artist data: http status: 404, code: -1 - https://api.spotify.com/v1/artists/3TVXtAsR1Inumwj472S9r4/related-artists:
 Not Found, reason: None
   ✅ 'Drake' EXISTS


TypeError: 'NoneType' object is not subscriptable